[Human Traffic Of Stadium - Leetcode](https://leetcode.com/problems/human-traffic-of-stadium/description/)

In [13]:
import findspark
findspark.init()

import datetime

In [14]:
from pyspark.sql.session import SparkSession
import pyspark.sql.functions as F
import pyspark.sql.types as T
from pyspark.sql.window import Window

spark = SparkSession.builder.master("local").appName("playground").getOrCreate()

In [15]:
stadium_schema = T.StructType(
    [
        T.StructField("id", T.IntegerType(), False),
        T.StructField("visit_date", T.DateType(), False),
        T.StructField("people", T.IntegerType(), False)
    ]
)

stadium_data = [
    (1, datetime.date(2017, 1, 1), 10),
    (2, datetime.date(2017, 1, 2), 109),
    (3, datetime.date(2017, 1, 3), 150),
    (4, datetime.date(2017, 1, 4), 99),
    (5, datetime.date(2017, 1, 5), 145),
    (6, datetime.date(2017, 1, 6), 1455),
    (7, datetime.date(2017, 1, 7), 199),
    (8, datetime.date(2017, 1, 9), 188)
]

stadium = spark.createDataFrame(stadium_data, stadium_schema)

stadium.show()

+---+----------+------+
| id|visit_date|people|
+---+----------+------+
|  1|2017-01-01|    10|
|  2|2017-01-02|   109|
|  3|2017-01-03|   150|
|  4|2017-01-04|    99|
|  5|2017-01-05|   145|
|  6|2017-01-06|  1455|
|  7|2017-01-07|   199|
|  8|2017-01-09|   188|
+---+----------+------+



In [16]:
window_spec = Window.partitionBy().orderBy("id")

stadium = (stadium.withColumn("day_before_yesterday", F.lag("people", 2).over(window_spec))
                  .withColumn("yesterday", F.lag("people", 1).over(window_spec))
                  .withColumn("day_after_tomorrow", F.lead("people", 2).over(window_spec))
                  .withColumn("tomorrow", F.lead("people", 1).over(window_spec)))

stadium.show()

+---+----------+------+--------------------+---------+------------------+--------+
| id|visit_date|people|day_before_yesterday|yesterday|day_after_tomorrow|tomorrow|
+---+----------+------+--------------------+---------+------------------+--------+
|  1|2017-01-01|    10|                null|     null|               150|     109|
|  2|2017-01-02|   109|                null|       10|                99|     150|
|  3|2017-01-03|   150|                  10|      109|               145|      99|
|  4|2017-01-04|    99|                 109|      150|              1455|     145|
|  5|2017-01-05|   145|                 150|       99|               199|    1455|
|  6|2017-01-06|  1455|                  99|      145|               188|     199|
|  7|2017-01-07|   199|                 145|     1455|              null|     188|
|  8|2017-01-09|   188|                1455|      199|              null|    null|
+---+----------+------+--------------------+---------+------------------+--------+



In [17]:
stadium = (stadium
           .where("""(people >= 100) AND (((tomorrow >= 100) AND (day_after_tomorrow >= 100)) 
                                        OR ((yesterday >= 100) AND (day_before_yesterday >= 100))
                                        OR ((yesterday >= 100) AND (tomorrow >= 100)))"""))

stadium.show()

+---+----------+------+--------------------+---------+------------------+--------+
| id|visit_date|people|day_before_yesterday|yesterday|day_after_tomorrow|tomorrow|
+---+----------+------+--------------------+---------+------------------+--------+
|  5|2017-01-05|   145|                 150|       99|               199|    1455|
|  6|2017-01-06|  1455|                  99|      145|               188|     199|
|  7|2017-01-07|   199|                 145|     1455|              null|     188|
|  8|2017-01-09|   188|                1455|      199|              null|    null|
+---+----------+------+--------------------+---------+------------------+--------+



In [18]:
stadium = stadium.selectExpr("id", "visit_date", "people")

stadium.show()

+---+----------+------+
| id|visit_date|people|
+---+----------+------+
|  5|2017-01-05|   145|
|  6|2017-01-06|  1455|
|  7|2017-01-07|   199|
|  8|2017-01-09|   188|
+---+----------+------+

